# Flow types

In the [previous notebook](https://github.com/monash-emu/summer/blob/master/docs/textbook/01-basic-model.ipynb), we introduced our general approach to creating and running a simple compartmental model of the transmission of an acute immunising infection. This basic model had only three compartments, which were linked together by two transition flows, with one additional flow through which the population simulated could exit the system (deaths). However, there are several epidemiological issues to consider when adding flows to the compartmental structure, and many different types of flow we might want to apply to our compartments.

_summer_'s [CompartmentalModel](http://summerepi.com/api/model.html) class offers a variety of intercompartmental flows that can be used to define the dynamics of a model. These consist of:

- [Transition flow](#Transition-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_transition_flow))
- [Infection density flow](#Infection-density-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_infection_density_flow))
- [Infection frequency flow](#Infection-frequency-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_infection_frequency_flow))
- [Death flow](#Death-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_death_flow))
- [Universal death flows](#Universal-death-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_universal_death_flows))
- [Crude birth flow](#Crude-birth-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_crude_birth_flow))
- [Importation flow](#Importation-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_importation_flow))
- [Replacement birth flow](#Replacement-birth-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_replacement_birth_flow))
- [Function flow](#Function-flow) ([docs](http://summerepi.com/api/model.html#summer.model.CompartmentalModel.add_function_flow))

Note that this is a somewhat "non-traditional" way of thinking about compartmental models, which are more often represented as ODEs (as mentioned in notebook 01). When defined in this way, a "flow" can be thought of an entry to or exit from a compartment, that increments/decrements the value of that compartment at a certain rate and is indicated with an addition/subtraction term in the ODE for that compartment. Each entry/exit to/from a compartment may have a corresponding exit/entry from/to a linked compartment. However, we believe that modellers more often think in terms of "compartments" linked by "flows", which is also reflected in the common illustration of such systems through flow diagrams.

Let's start off with a small amount of boilerplate (repeated) code that we'll need for just about all our notebooks.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi2
except:
  IN_COLAB = False

In [ ]:
import pandas as pd
from jax import numpy as jnp
from typing import Dict
import copy

from summer2 import CompartmentalModel
from summer2.parameters import Parameter as param
from summer2.parameters import Function as func
from summer2.parameters import Time

pd.options.plotting.backend = "plotly"

In [ ]:
def get_base_model(
    parameters: Dict
) -> CompartmentalModel:
    """
    Generate an instance of a very simple two-compartment model that allows transition
    from one of the two states to the other through a single flow linking them, 
    population distribution and parameters hard-coded.
    This is just about the simplest "closed" system possible,
    such that we're tracking where our starting population goes to and how many
    are left in their origin compartment.
    
    Returns:
        The summer model object
    """
    compartments = (
        "healthy",
        "diseased",
    )
    analysis_times = (0, 20)
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=[],
    )
    model.set_initial_population(
        distribution={"healthy": 1.}
    )
    model.add_transition_flow(
        "onset", 
        fractional_rate=param("onset_rate"), 
        source="healthy", 
        dest="diseased"
    )
    return model

## Transition flow

With this type of flow, people in the source compartment transition to the destination compartment at a _per capita_ rate defined by the flow's parameter.
By "_per capita_" we mean that the rate of this flow is calculated as the product of the parameter value assigned to the flow rate
and the size of the population in the source compartment.

With just a single outflow from the source compartment, 
the sojourn time is the reciprocal of the rate of outflow.
(By competing flows, we mean flows that have the same source compartment.)
Where multiple flows share a common source compartment, the sojourn time is the reciprocal of the sum of the outflows from that compartment.

In the example below, we will add a progression flow called "onset" where the average time to developing disease is 0.1 time units.
This can be thought of as the mean time to disease onset for someone in the healthy category as being 10 time units.

In [ ]:
transition_params = {
    "population": 1.,
    "onset_rate": 0.1,
}

transition_model = get_base_model(transition_params)

transition_model.run(parameters=transition_params)
compartment_values = transition_model.get_outputs_df()
compartment_values.plot()

### Time-varying transition flow

The rate at which people transition can be set as a constant, or it can be defined as a function of time. This is the case for all _summer_ flows: every parameter can be a constant _or_ a function of time.
Parameters also take a `computed_values` argument, which is a dictionary of values computed at runtime that is not specific to any individual flow and can be specified by the user.
We'll come back to examples of this later, but for now this is just an example of how to create a function that _summer_ can understand and can act on the `time` variable within its scope.
Older versions of _summer_ do not include the computed values argument, and it is a good idea to provide a default of `None` for functions in which this argument is not used (i.e. depend solely on time).

In [ ]:
def get_step_onset_model(
    parameters: Dict
) -> CompartmentalModel:
    """
    Generates a model that is almost as simple as the previous example,
    but the parameter varies according to a step function.
    
    Returns:
        The summer model object
    """
    compartments = (
        "healthy",
        "diseased",
    )
    analysis_times = (0, 20)
    
    def scaled_onset_rate(time):
        return jnp.where(
            time < 10., 
            parameters["onset_rate"], 
            0.
        )

    onset_rate = func(scaled_onset_rate, [Time])
   
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=[],
    )
    model.set_initial_population(
        distribution={"healthy": 1.}
    )
    model.add_transition_flow(
        "onset",
        fractional_rate=onset_rate,
        source="healthy", 
        dest="diseased"
    )
    return model

In [ ]:
step_model = get_step_onset_model(transition_params)
step_model.run(parameters=transition_params)
compartment_values = step_model.get_outputs_df()
compartment_values.plot()

Of course, this function is not intended to represent anything very epidemiologically plausible,
but the syntax could be reused with a more complicated function used instead.
Also to stress, this syntax can be used for the other flow types we will now discuss.

In [ ]:
sir_params = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
}

sir_freq_model = get_sir_freq_model(sir_params)
sir_freq_model.run(parameters=sir_params)
compartment_values = sir_freq_model.get_outputs_df()
compartment_values.plot()

We are now beginning to see some interesting dynamics, although the model is still simpler than the one we already saw in Notebook 1.

## Death flow

With a death flow, some percent of people in a user-selected source compartment die and leave the system every time unit.

In [ ]:
def get_sir_with_death_model(
    parameters: Dict
)-> CompartmentalModel:
    """
    Take the previous SIR model and add a death flow.
    This is now structurally similar to the model we introduced in Notebook 1.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    model = get_sir_freq_model(parameters)

    # Add the death rate to the model
    model.add_death_flow("infection_death", death_rate=param("death_rate"), source="infectious")
    
    return model

In [ ]:
sir_params.update({"death_rate": 0.3})  # Add an arbitrary parameter value

sir_death_model = get_sir_with_death_model(sir_params)
    
sir_death_model.run(parameters=sir_params)
compartment_values = sir_death_model.get_outputs_df()
compartment_values.plot()

## Universal death flow

Adding "universal deaths" is a convenient way to set up a death flow for every compartment, which can account for non-disease mortality (e.g. heart disease, trauma, etc.). This is functionally the same as manually adding a death flow for every compartment with the same parameter value. You can adjust the universal death rate for particular strata later during the stratification process (e.g. for age-specific mortality rates).

In [ ]:
def get_sir_with_popdeath_model(
    parameters: Dict
)-> CompartmentalModel:
    """
    Take the previous SIR model and add a universal or population-wide death flow.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    model = get_sir_freq_model(parameters)

    # Add the population death rate to the model
    model.add_universal_death_flows("universal_death", death_rate=param("pop_death"))
    
    return model

In [ ]:
sir_params.update({"pop_death": 0.05})  # Add an arbitrary population-wide death rate

sir_pop_death_model = get_sir_with_popdeath_model(sir_params)
    
sir_pop_death_model.run(parameters=sir_params)
compartment_values = sir_pop_death_model.get_outputs_df()
compartment_values.plot()

# Please ignore the following deprecation warning, we will remove this in 
# the next summer release.

## Importation flow

An absolute number of people arrive in the destination per time unit. This can be used to model arrivals from outside of the modelled region.

'split_imports' determines whether this number is split over the existing destination compartments (`True`), or the full number of people sent to each (`False`).  In this example the behaviour is the same (since the destination of the flow is a single compartment), but for stratified models, this can be an important distinction - we will cover this in more detail in later notebooks.

In [ ]:
def get_sir_with_imports_model(
    parameters: Dict
)-> CompartmentalModel:
    """
    Take the previous SIR model and add an importation flow.
    
    Args:
        parameters: The parameter values to be used in running the model
    Returns:
        The summer model object
    """
    
    model = get_sir_freq_model(parameters)

    # Add importations to the model
    model.add_importation_flow(
        "imports", 
        num_imported=param("imports"), 
        dest="susceptible", 
        split_imports=True
    )
    model.add_importation_flow(
        "imports", 
        num_imported=param("imports"), 
        dest="recovered", 
        split_imports=True
    )
    
    return model

In [ ]:
sir_params.update({"imports": 15.})  # Add an arbitrary importation rate

sir_imports_model = get_sir_with_imports_model(sir_params)
    
sir_imports_model.run(parameters=sir_params)
compartment_values = sir_imports_model.get_outputs_df()
compartment_values.plot()

The population size increases quickly with this arbitrary parameter choice.

As expected, we end up with more of the population in the susceptible compartment, although this model is now a "closed" system, in that it retains a constant total population size throughout.

## Summary
We've worked through several of the most commonly used flow types that we would use in infectious diseases modelling, representing these in _summer_ code and and showing their epidemiological effects. Of course, flows will be fundamental to the models we illustrate in the following notebooks as well, so we hope you have a good grip on the general syntax now.

A detailed API reference of the CompartmentalModel class can be found [here](http://summerepi.com/api/model.html)